In [ ]:
import cv2 as cv
import numpy as np

# Initialize webcam
cap = cv.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Convert BGR to HSV
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # Define range for yellow color in HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([60, 255, 255])  # Ensure upper > lower

    # Threshold to extract yellow objects
    mask = cv.inRange(hsv, lower_yellow, upper_yellow)

    # Bitwise-AND mask with original image
    res = cv.bitwise_and(frame, frame, mask=mask)
    
    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    contour_count = 0  # Initialize a counter for contours

    for contour in contours:
        if cv.contourArea(contour) > 500:  # Filter out small areas
            contour_count += 1  # Increment the counter
            (x, y), radius = cv.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            cv.circle(frame, center, radius, (0, 255, 0), 2)  # Draw bounding circle

    # Display contour count on the frame
    font = cv.FONT_HERSHEY_SIMPLEX
    cv.putText(frame, f'Object Detected: {contour_count}', (10, 30), font, 1, (0, 255, 0), 2, cv.LINE_AA)

    # Display windows
    cv.imshow('Original Frame', frame)
    cv.imshow('Mask', mask)
    cv.imshow('Filtered Output', res)

    # Press 'Esc' to exit
    if cv.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv.destroyAllWindows()
